In [1]:
import pandas as pd
import plotly.express as px
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, TextVectorization, Embedding
from tensorflow.keras.models import Model

In [ ]:
def data_processing(data_path: str) -> pd.DataFrame:
    """
    Read the input data, and transform into the structure needed for training a model
    
    Inputs:
        data_path:
            A string path to the input Mr Boston Cocktail Dataset
        
    Outputs:
        data:
            A pandas dataframe with all of the needed columns for training
    """
    
    

In [2]:
df = pd.read_csv("mr-boston-flattened.csv")
df

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,1,"Lime wedge, superfine sugar",1 oz,Gin,1 oz,Triple Sec,1 oz,Fresh Lime Juice,NaN,NaN,NaN,NaN,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces
986,Minnehaha Cocktail,Cocktail Classics,1/4 oz,Juice of Orange,1/2 oz,Dry Vermouth,1/2 oz,Sweet Vermouth,1 oz,Old Mr. Boston Dry Gin,NaN,NaN,NaN,NaN,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces
987,Wallick Cocktail,Gin,1 1/2 oz,Gin,1 1/2 oz,Dry Vermouth,1 oz,Triple Sec,NaN,NaN,NaN,NaN,NaN,NaN,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces
988,Waikiki Beachcomber,Gin,3/4 oz,Gin,3/4 oz,Triple Sec,1/2 oz,Pineapple Juice,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces


In [3]:
textvec = TextVectorization(
    max_tokens=10000,
    output_sequence_length=5,
    pad_to_max_tokens=True
)

textvec.adapt(df["name"])

voc = textvec.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [32]:
voc

['',
 '[UNK]',
 'cocktail',
 'the',
 'fizz',
 'no',
 'rum',
 'martini',
 'gin',
 'tequila',
 'sour',
 'vodka',
 'flip',
 'and',
 'punch',
 'highball',
 'collins',
 '1',
 'rose',
 'cooler',
 'brandy',
 'white',
 'pink',
 'golden',
 'cherry',
 'whiskey',
 'special',
 'orange',
 'bourbon',
 'apple',
 'milk',
 'margarita',
 'lady',
 '2',
 'purple',
 'of',
 'la',
 'irish',
 'dry',
 'daisy',
 'tea',
 'saronno',
 'rickey',
 'red',
 'green',
 'club',
 'chocolate',
 'blue',
 'black',
 'velvet',
 'sling',
 'silver',
 'peach',
 'mint',
 'lemonade',
 'grasshopper',
 'el',
 'coffee',
 'bronx',
 'boston',
 'blossom',
 'amaretto',
 'whisky',
 'tnt',
 'sunrise',
 'street',
 'sherry',
 'screwdriver',
 'royal',
 'pineapple',
 'passion',
 'oldfashioned',
 'kiss',
 'julep',
 'jamaican',
 'jack',
 'grapefruit',
 'gimlet',
 'flying',
 'dubonnet',
 'dream',
 'de',
 'cuban',
 'canadian',
 'bull',
 'bloody',
 'apricot',
 'a',
 'wine',
 'widows',
 'vermouth',
 'thunder',
 'taylor',
 'swizzle',
 'stone',
 'sting

In [4]:
path_to_glove_file = "glove.6B.300d.txt"


embeddings_index = {}
with open(path_to_glove_file, encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [5]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 985 words (90 misses)


In [6]:
ingredients = pd.concat([df["ingredient-1"], df["ingredient-2"], df["ingredient-3"], df["ingredient-4"], df["ingredient-5"], df["ingredient-6"]]).unique()
ingredients.shape

from sklearn.preprocessing import LabelEncoder

ingredient_encoder = LabelEncoder()
ingredient_encoder.fit(ingredients)

LabelEncoder()

In [7]:
df["ingredient-1"] = ingredient_encoder.transform(df["ingredient-1"])
df["ingredient-2"] = ingredient_encoder.transform(df["ingredient-2"])
df["ingredient-3"] = ingredient_encoder.transform(df["ingredient-3"])
df["ingredient-4"] = ingredient_encoder.transform(df["ingredient-4"])
df["ingredient-5"] = ingredient_encoder.transform(df["ingredient-5"])
df["ingredient-6"] = ingredient_encoder.transform(df["ingredient-6"])

In [8]:
df

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,325,1 oz,445,1 oz,297,1 oz,334,NaN,687,NaN,687,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,325,1/2 oz,551,1/4 oz,289,1/4 oz,292,NaN,687,NaN,687,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,35,1 oz,113,NaN,37,NaN,687,NaN,687,NaN,687,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,292,1/2 oz,479,2 oz,325,NaN,687,NaN,687,NaN,687,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,169,2 oz,142,2 oz,469,1 oz,428,1 oz,530,NaN,687,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,1,660,1 oz,234,1 oz,568,1 oz,193,NaN,687,NaN,687,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces
986,Minnehaha Cocktail,Cocktail Classics,1/4 oz,289,1/2 oz,178,1/2 oz,551,1 oz,396,NaN,687,NaN,687,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces
987,Wallick Cocktail,Gin,1 1/2 oz,234,1 1/2 oz,178,1 oz,568,NaN,687,NaN,687,NaN,687,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces
988,Waikiki Beachcomber,Gin,3/4 oz,234,3/4 oz,568,1/2 oz,469,NaN,687,NaN,687,NaN,687,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces


In [9]:
measurements = pd.concat([df["measurement-1"], df["measurement-2"], df["measurement-3"], df["measurement-4"], df["measurement-5"], df["measurement-6"]]).unique()
measurements.shape

measurement_encoder = LabelEncoder()
measurement_encoder.fit(measurements)

LabelEncoder()

In [10]:
df["measurement-1"] = measurement_encoder.transform(df["measurement-1"])
df["measurement-2"] = measurement_encoder.transform(df["measurement-2"])
df["measurement-3"] = measurement_encoder.transform(df["measurement-3"])
df["measurement-4"] = measurement_encoder.transform(df["measurement-4"])
df["measurement-5"] = measurement_encoder.transform(df["measurement-5"])
df["measurement-6"] = measurement_encoder.transform(df["measurement-6"])

In [11]:
pieces = []
pieces.append(df[["name"]])

for i in range(6):
    temp = pd.DataFrame(data=tf.keras.utils.to_categorical(df[f"ingredient-{i+1}"]), columns=[f"I{i+1} - {c}" for c in ingredient_encoder.classes_])
    pieces.append(temp)
    
    temp = pd.DataFrame(data=tf.keras.utils.to_categorical(df[f"measurement-{i+1}"]), columns=[f"M{i+1} - {c}" for c in measurement_encoder.classes_])
    pieces.append(temp)

data = pd.concat(pieces, axis=1)
data

,name,I1 - Old Mr. Boston Triple Sec,I1 - 100-proof Vodka,I1 - 151-Proof Rum,I1 - 17-year-old J. Wray and Nephew Ltd. Rum,I1 - 7-Up,I1 - Absinthe,I1 - Absinthe Substitute,I1 - Absinthe or pastis,I1 - Acai berry flavored vodka,...,M6 - 64 Ginger ale,M6 - 7,M6 - 8 Milk,M6 - For glass,M6 - champagne,M6 - fruits in season,M6 - spirals of lemon and orange peel,M6 - splash,M6 - twist of lemon,M6 - nan
0,Gauguin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Fort Lauderdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Apple Pie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Cuban Cocktail No. 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Cool Carlos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
986,Minnehaha Cocktail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
987,Wallick Cocktail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
988,Waikiki Beachcomber,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
encoder_inputs = Input(shape=(1,), dtype=tf.string)

e = textvec(encoder_inputs)

e = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)(e)

encoder_outputs = LSTM(128)(e)

d = Dense(64)(encoder_outputs)
d = Dropout(0.25)(d)
d = Dense(64)(d)
d = Dropout(0.25)(d)

i1 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i1_output")(d)
i2 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i2_output")(d)
i3 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i3_output")(d)
i4 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i4_output")(d)
i5 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i5_output")(d)
i6 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i6_output")(d)

m1 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m1_output")(d)
m2 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m2_output")(d)
m3 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m3_output")(d)
m4 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m4_output")(d)
m5 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m5_output")(d)
m6 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m6_output")(d)

decoder_outputs = [i1, m1, i2, m2, i3, m3, i4, m4, i5, m5, i6, m6]
model = Model(encoder_inputs, decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 5)           0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 5, 300)       323100      ['text_vectorization[0][0]']     
                                                                                                  
 lstm (LSTM)                    (None, 128)          219648      ['embedding[0][0]']          

In [13]:
losses = {
    "i1_output":"categorical_crossentropy",
    "i2_output":"categorical_crossentropy",
    "i3_output":"categorical_crossentropy",
    "i4_output":"categorical_crossentropy",
    "i5_output":"categorical_crossentropy",
    "i6_output":"categorical_crossentropy",
    "m1_output":"categorical_crossentropy",
    "m2_output":"categorical_crossentropy",
    "m3_output":"categorical_crossentropy",
    "m4_output":"categorical_crossentropy",
    "m5_output":"categorical_crossentropy",
    "m6_output":"categorical_crossentropy",
}

optimizer='adam'
metrics = {
    "i1_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "i2_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "i3_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "i4_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "i5_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "i6_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "m1_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "m2_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "m3_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "m4_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "m5_output":tf.metrics.CategoricalAccuracy(name='acc'),
    "m6_output":tf.metrics.CategoricalAccuracy(name='acc'),
}

model.compile(loss=losses, metrics=metrics, optimizer=optimizer)

In [14]:
X = data[["name"]].values

Ys = {}

for i in range(6):
    temp = data[[col for col in data.columns if f"I{i+1} - " in col]]
    Ys[f"i{i+1}_output"] = temp.values
    if temp.values.shape[1] != len(ingredient_encoder.classes_):
        raise RuntimeError(f"Too many columns pulled! Problem on I{i+1}")
    
    temp = data[[col for col in data.columns if f"M{i+1} - " in col]]
    Ys[f"m{i+1}_output"] = temp.values
    if temp.values.shape[1] != len(measurement_encoder.classes_):
        raise RuntimeError(f"Too many columns pulled! Problem on I{i+1}")

model.fit(X, Ys, epochs=100, verbose=1)

Epoch 1/100
31/31 [==============================] - 6s 41ms/step - loss: 54.2080 - i1_output_loss: 6.1165 - m1_output_loss: 3.8432 - i2_output_loss: 6.1898 - m2_output_loss: 3.8368 - i3_output_loss: 6.0910 - m3_output_loss: 3.7422 - i4_output_loss: 5.5098 - m4_output_loss: 3.8085 - i5_output_loss: 4.8679 - m5_output_loss: 3.2489 - i6_output_loss: 4.5657 - m6_output_loss: 2.3877 - i1_output_acc: 0.0253 - m1_output_acc: 0.1798 - i2_output_acc: 0.0141 - m2_output_acc: 0.1960 - i3_output_acc: 0.0121 - m3_output_acc: 0.1535 - i4_output_acc: 0.1384 - m4_output_acc: 0.1687 - i5_output_acc: 0.3596 - m5_output_acc: 0.4980 - i6_output_acc: 0.4606 - m6_output_acc: 0.7717
Epoch 2/100
31/31 [==============================] - 2s 61ms/step - loss: 34.6338 - i1_output_loss: 4.7658 - m1_output_loss: 2.4662 - i2_output_loss: 5.0687 - m2_output_loss: 2.4187 - i3_output_loss: 4.8440 - m3_output_loss: 2.4266 - i4_output_loss: 3.7225 - m4_output_loss: 2.3914 - i5_output_loss: 2.5137 - m5_output_loss: 1.776

31/31 [==============================] - 2s 61ms/step - loss: 28.5630 - i1_output_loss: 4.1956 - m1_output_loss: 1.9760 - i2_output_loss: 4.6227 - m2_output_loss: 1.9146 - i3_output_loss: 4.3878 - m3_output_loss: 1.8880 - i4_output_loss: 3.1525 - m4_output_loss: 1.8659 - i5_output_loss: 1.9018 - m5_output_loss: 1.2349 - i6_output_loss: 1.0781 - m6_output_loss: 0.3450 - i1_output_acc: 0.1566 - m1_output_acc: 0.3141 - i2_output_acc: 0.0899 - m2_output_acc: 0.2778 - i3_output_acc: 0.1162 - m3_output_acc: 0.3030 - i4_output_acc: 0.4091 - m4_output_acc: 0.4444 - i5_output_acc: 0.6768 - m5_output_acc: 0.6990 - i6_output_acc: 0.8394 - m6_output_acc: 0.9273
Epoch 14/100
31/31 [==============================] - 2s 62ms/step - loss: 27.9845 - i1_output_loss: 4.1168 - m1_output_loss: 1.9358 - i2_output_loss: 4.5297 - m2_output_loss: 1.8782 - i3_output_loss: 4.3145 - m3_output_loss: 1.8452 - i4_output_loss: 3.1050 - m4_output_loss: 1.8403 - i5_output_loss: 1.8607 - m5_output_loss: 1.2170 - i6_outp

31/31 [==============================] - 2s 66ms/step - loss: 22.5632 - i1_output_loss: 3.3425 - m1_output_loss: 1.6371 - i2_output_loss: 3.7646 - m2_output_loss: 1.5664 - i3_output_loss: 3.5127 - m3_output_loss: 1.5092 - i4_output_loss: 2.4434 - m4_output_loss: 1.4070 - i5_output_loss: 1.4252 - m5_output_loss: 0.9029 - i6_output_loss: 0.8031 - m6_output_loss: 0.2491 - i1_output_acc: 0.2202 - m1_output_acc: 0.4222 - i2_output_acc: 0.1495 - m2_output_acc: 0.4263 - i3_output_acc: 0.1889 - m3_output_acc: 0.4596 - i4_output_acc: 0.4556 - m4_output_acc: 0.5545 - i5_output_acc: 0.6970 - m5_output_acc: 0.7566 - i6_output_acc: 0.8414 - m6_output_acc: 0.9313
Epoch 26/100
31/31 [==============================] - 2s 73ms/step - loss: 22.1351 - i1_output_loss: 3.2759 - m1_output_loss: 1.5981 - i2_output_loss: 3.6834 - m2_output_loss: 1.5491 - i3_output_loss: 3.4512 - m3_output_loss: 1.4681 - i4_output_loss: 2.3875 - m4_output_loss: 1.3872 - i5_output_loss: 1.3400 - m5_output_loss: 0.9105 - i6_outp

31/31 [==============================] - 2s 62ms/step - loss: 17.3547 - i1_output_loss: 2.4623 - m1_output_loss: 1.3040 - i2_output_loss: 2.8578 - m2_output_loss: 1.2907 - i3_output_loss: 2.7021 - m3_output_loss: 1.2456 - i4_output_loss: 1.8363 - m4_output_loss: 1.1347 - i5_output_loss: 0.9663 - m5_output_loss: 0.7773 - i6_output_loss: 0.5480 - m6_output_loss: 0.2297 - i1_output_acc: 0.3727 - m1_output_acc: 0.5323 - i2_output_acc: 0.2586 - m2_output_acc: 0.5242 - i3_output_acc: 0.2970 - m3_output_acc: 0.5495 - i4_output_acc: 0.5152 - m4_output_acc: 0.6131 - i5_output_acc: 0.7596 - m5_output_acc: 0.7687 - i6_output_acc: 0.8737 - m6_output_acc: 0.9343
Epoch 38/100
31/31 [==============================] - 2s 61ms/step - loss: 16.8913 - i1_output_loss: 2.3407 - m1_output_loss: 1.2496 - i2_output_loss: 2.8021 - m2_output_loss: 1.2655 - i3_output_loss: 2.6545 - m3_output_loss: 1.2079 - i4_output_loss: 1.8088 - m4_output_loss: 1.1389 - i5_output_loss: 0.9579 - m5_output_loss: 0.7677 - i6_outp

31/31 [==============================] - 2s 62ms/step - loss: 13.7618 - i1_output_loss: 1.8641 - m1_output_loss: 1.0953 - i2_output_loss: 2.2488 - m2_output_loss: 1.0842 - i3_output_loss: 2.0742 - m3_output_loss: 1.0566 - i4_output_loss: 1.4068 - m4_output_loss: 1.0017 - i5_output_loss: 0.7141 - m5_output_loss: 0.6774 - i6_output_loss: 0.3607 - m6_output_loss: 0.1779 - i1_output_acc: 0.4818 - m1_output_acc: 0.6263 - i2_output_acc: 0.4051 - m2_output_acc: 0.5960 - i3_output_acc: 0.4283 - m3_output_acc: 0.6081 - i4_output_acc: 0.6162 - m4_output_acc: 0.6646 - i5_output_acc: 0.7960 - m5_output_acc: 0.7859 - i6_output_acc: 0.9071 - m6_output_acc: 0.9394
Epoch 50/100
31/31 [==============================] - 2s 61ms/step - loss: 13.5560 - i1_output_loss: 1.8603 - m1_output_loss: 1.0715 - i2_output_loss: 2.1362 - m2_output_loss: 1.0830 - i3_output_loss: 2.0772 - m3_output_loss: 1.0737 - i4_output_loss: 1.3902 - m4_output_loss: 0.9569 - i5_output_loss: 0.6958 - m5_output_loss: 0.6382 - i6_outp

31/31 [==============================] - 2s 62ms/step - loss: 11.6599 - i1_output_loss: 1.5453 - m1_output_loss: 0.9623 - i2_output_loss: 1.8193 - m2_output_loss: 0.9809 - i3_output_loss: 1.7698 - m3_output_loss: 0.9685 - i4_output_loss: 1.1439 - m4_output_loss: 0.8613 - i5_output_loss: 0.5674 - m5_output_loss: 0.5727 - i6_output_loss: 0.2850 - m6_output_loss: 0.1836 - i1_output_acc: 0.5646 - m1_output_acc: 0.6515 - i2_output_acc: 0.4909 - m2_output_acc: 0.6465 - i3_output_acc: 0.4879 - m3_output_acc: 0.6313 - i4_output_acc: 0.6848 - m4_output_acc: 0.6949 - i5_output_acc: 0.8343 - m5_output_acc: 0.8202 - i6_output_acc: 0.9283 - m6_output_acc: 0.9434
Epoch 62/100
31/31 [==============================] - 2s 63ms/step - loss: 11.3383 - i1_output_loss: 1.5577 - m1_output_loss: 0.9552 - i2_output_loss: 1.7149 - m2_output_loss: 0.9647 - i3_output_loss: 1.6910 - m3_output_loss: 0.9554 - i4_output_loss: 1.1309 - m4_output_loss: 0.8469 - i5_output_loss: 0.5634 - m5_output_loss: 0.5362 - i6_outp

31/31 [==============================] - 2s 60ms/step - loss: 9.9887 - i1_output_loss: 1.3345 - m1_output_loss: 0.8760 - i2_output_loss: 1.4301 - m2_output_loss: 0.8987 - i3_output_loss: 1.4757 - m3_output_loss: 0.8822 - i4_output_loss: 0.9290 - m4_output_loss: 0.7722 - i5_output_loss: 0.5063 - m5_output_loss: 0.4970 - i6_output_loss: 0.2225 - m6_output_loss: 0.1645 - i1_output_acc: 0.6131 - m1_output_acc: 0.6859 - i2_output_acc: 0.5848 - m2_output_acc: 0.6687 - i3_output_acc: 0.5717 - m3_output_acc: 0.6667 - i4_output_acc: 0.7293 - m4_output_acc: 0.7232 - i5_output_acc: 0.8606 - m5_output_acc: 0.8354 - i6_output_acc: 0.9414 - m6_output_acc: 0.9515
Epoch 74/100
31/31 [==============================] - 2s 61ms/step - loss: 9.9199 - i1_output_loss: 1.3035 - m1_output_loss: 0.8977 - i2_output_loss: 1.4365 - m2_output_loss: 0.8744 - i3_output_loss: 1.4506 - m3_output_loss: 0.8700 - i4_output_loss: 0.9704 - m4_output_loss: 0.7884 - i5_output_loss: 0.4501 - m5_output_loss: 0.4860 - i6_output

31/31 [==============================] - 2s 63ms/step - loss: 9.1380 - i1_output_loss: 1.1427 - m1_output_loss: 0.8354 - i2_output_loss: 1.2668 - m2_output_loss: 0.8476 - i3_output_loss: 1.2619 - m3_output_loss: 0.8615 - i4_output_loss: 0.8503 - m4_output_loss: 0.7651 - i5_output_loss: 0.4570 - m5_output_loss: 0.4389 - i6_output_loss: 0.2450 - m6_output_loss: 0.1658 - i1_output_acc: 0.6626 - m1_output_acc: 0.7000 - i2_output_acc: 0.6182 - m2_output_acc: 0.6980 - i3_output_acc: 0.6303 - m3_output_acc: 0.6586 - i4_output_acc: 0.7545 - m4_output_acc: 0.7364 - i5_output_acc: 0.8646 - m5_output_acc: 0.8515 - i6_output_acc: 0.9343 - m6_output_acc: 0.9485
Epoch 86/100
31/31 [==============================] - 2s 61ms/step - loss: 8.8913 - i1_output_loss: 1.1282 - m1_output_loss: 0.8046 - i2_output_loss: 1.2641 - m2_output_loss: 0.8069 - i3_output_loss: 1.2542 - m3_output_loss: 0.8168 - i4_output_loss: 0.8304 - m4_output_loss: 0.7087 - i5_output_loss: 0.4215 - m5_output_loss: 0.4513 - i6_output

31/31 [==============================] - 2s 61ms/step - loss: 8.1745 - i1_output_loss: 1.0503 - m1_output_loss: 0.7318 - i2_output_loss: 1.1239 - m2_output_loss: 0.7462 - i3_output_loss: 1.1345 - m3_output_loss: 0.8073 - i4_output_loss: 0.7324 - m4_output_loss: 0.6827 - i5_output_loss: 0.4058 - m5_output_loss: 0.4188 - i6_output_loss: 0.2074 - m6_output_loss: 0.1333 - i1_output_acc: 0.6960 - m1_output_acc: 0.7505 - i2_output_acc: 0.6717 - m2_output_acc: 0.7202 - i3_output_acc: 0.6616 - m3_output_acc: 0.6970 - i4_output_acc: 0.7758 - m4_output_acc: 0.7576 - i5_output_acc: 0.8778 - m5_output_acc: 0.8626 - i6_output_acc: 0.9475 - m6_output_acc: 0.9566
Epoch 98/100
31/31 [==============================] - 2s 63ms/step - loss: 8.1670 - i1_output_loss: 1.0171 - m1_output_loss: 0.7442 - i2_output_loss: 1.0904 - m2_output_loss: 0.7742 - i3_output_loss: 1.0919 - m3_output_loss: 0.7819 - i4_output_loss: 0.7558 - m4_output_loss: 0.6931 - i5_output_loss: 0.4197 - m5_output_loss: 0.4426 - i6_output

In [17]:
len(predicted_ingredients)

12

In [37]:
def create_recipe(name: str):
    """
    
    """
    
    predicted_ingredients = model.predict([name])
    
    recipe = []
    for i in range(6):
        ingredient = predicted_ingredients[2*i]
        measurement = predicted_ingredients[2*i + 1]
        
        ingredient = np.argmax(ingredient)
        measurement = np.argmax(measurement)
        
        ingredient = ingredient_encoder.classes_[ingredient]
        measurement = measurement_encoder.classes_[measurement]
        
        if str(ingredient).strip().lower() == "nan":
            continue
        else:
            recipe.append(f"{measurement} of {ingredient}")
    
    return recipe

create_recipe("Lemon Sour")

['1 1/2 oz of  Lemon wedge, superfine sugar',
 '2 oz of  Gin',
 '1/4 oz of  Grenadine',
 '1/4 oz of  Simple Syrup']